In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!ls /content/gdrive/My\ Drive/eva-04/S9/* 

In [31]:
!cat '/content/gdrive/My Drive/eva-04/S9/models/QuizDNN.py'

from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

class QuizDNN(torch.nn.Module):
    def __init__(self, dropout_value = 0.1):
        """"Constructor of the class"""
        super(CIFAR_Depthwise_Separable_Model, self).__init__()
        # Input Block
        self.dropout_value = dropout_value
        #Depthwise separate 3 input channels into 3 output channels
        
        #self.conv1 = nn.Conv2d(3, 6, 5)  # RF= 
        #self.pool = nn.MaxPool2d(2, 2)
        #self.conv2 = nn.Conv2d(6, 16, 5)
        #self.fc1 = nn.Linear(16 * 5 * 5, 120)
        #self.fc2 = nn.Linear(120, 84)
        #self.fc3 = nn.Linear(84, 10)
        #torch.nn.Conv2d(in_channels, out_channels=k*in_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
        
        self.layer1_depthwise = nn.Conv2d(in_channels=3, out_channels=1*3, kern

In [0]:
!cat '/content/gdrive/My Drive/eva-04/S9/gradcam/gradcam_utils.py'

We'll need to update our path to import from Drive.

In [0]:
import sys
sys.path.append('/content/gdrive/My Drive/eva-04/S9/')

Now we can import the library and use the function.

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


In [10]:
from __future__ import print_function

from models import QuizDNN as quiz
from transformations import CIFARModel10_Alb as cifar
from datetime import datetime 
print("Current Date/Time: ", datetime.now())

import importlib
importlib.reload(quiz)
importlib.reload(cifar)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

SEED = 1
# CUDA?
print("CUDA Available?", use_cuda)

# For reproducibility
torch.manual_seed(SEED)

if use_cuda:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
#dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if use_cuda else dict(shuffle=True, batch_size=64)

# train dataloader
#train_loader = torch.utils.data.DataLoader(g_train, **dataloader_args)

# test dataloader
#test_loader = torch.utils.data.DataLoader(g_test, **dataloader_args)
#import images as im
#import random
#X,Y,names = im.get_data('cifar-10-batches-py/data_batch_1')
#import numpy as np
#import matplotlib.pyplot as plt
#import os
#importlib.reload(im)
#for x in range(100):
#  im.visualize_image(X,Y,names,random.randint(1,10000),'/content/gdrive/My Drive/eva-04/S9/images')


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Current Date/Time:  2020-03-21 20:57:43.263742
Files already downloaded and verified
Files already downloaded and verified
cuda
CUDA Available? True


SHOW PARAMETERS

RUNNING WITH REGULARIZATION

In [0]:
from datetime import datetime 
print("Current Date/Time: ", datetime.now())

from utils.global_fun import test_model
from utils.global_fun import train_model
from utils.global_fun import show_summary, run_model,  draw_accuracy_loss_change_graps

from models import QuizDNN as quiz
from transformations import CIFARModel10_Alb as cifar

import importlib
importlib.reload(quiz)
importlib.reload(cifar)

model = quiz.QuizDNN().to(device)
cifar_model = cifar.CIFARModel10_Alb(model)
show_summary(cifar_model,input_size=(3,32,32))
run_model(cifar_model, device, doL1= 1, doL2=1, LAMBDA=0.0005, EPOCHS=40)

ACCURACY , LOSS CHANGE GRAPHS


In [0]:
draw_accuracy_loss_change_graps(cifar_model,cifar_model,cifar_model,cifar_model)

GRADCAM VISUALIZATION

PRINT MODAL STATE DICT

In [0]:
for param_tensor in cifar_model.m_model.state_dict():
    print(param_tensor, "\t", cifar_model.m_model.state_dict()[param_tensor].size())

conv1.weight 	 torch.Size([64, 3, 3, 3])
bn1.weight 	 torch.Size([64])
bn1.bias 	 torch.Size([64])
bn1.running_mean 	 torch.Size([64])
bn1.running_var 	 torch.Size([64])
bn1.num_batches_tracked 	 torch.Size([])
layer1.0.conv1.weight 	 torch.Size([64, 64, 3, 3])
layer1.0.bn1.weight 	 torch.Size([64])
layer1.0.bn1.bias 	 torch.Size([64])
layer1.0.bn1.running_mean 	 torch.Size([64])
layer1.0.bn1.running_var 	 torch.Size([64])
layer1.0.bn1.num_batches_tracked 	 torch.Size([])
layer1.0.conv2.weight 	 torch.Size([64, 64, 3, 3])
layer1.0.bn2.weight 	 torch.Size([64])
layer1.0.bn2.bias 	 torch.Size([64])
layer1.0.bn2.running_mean 	 torch.Size([64])
layer1.0.bn2.running_var 	 torch.Size([64])
layer1.0.bn2.num_batches_tracked 	 torch.Size([])
layer1.1.conv1.weight 	 torch.Size([64, 64, 3, 3])
layer1.1.bn1.weight 	 torch.Size([64])
layer1.1.bn1.bias 	 torch.Size([64])
layer1.1.bn1.running_mean 	 torch.Size([64])
layer1.1.bn1.running_var 	 torch.Size([64])
layer1.1.bn1.num_batches_tracked 	 torch.

OPTIMIZER STATE DICT

In [0]:
for var_name in cifar_model.m_optimizer.state_dict():
    print(var_name, "\t", cifar_model.m_optimizer.state_dict()[var_name])

state 	 {140012712964168: {'momentum_buffer': tensor([[[[ 5.7262e-02,  3.9019e-02, -8.5817e-03],
          [ 7.6983e-02,  3.7400e-02, -1.9687e-03],
          [ 1.0333e-01,  1.2453e-01,  8.8726e-02]],

         [[-9.5121e-05, -8.6054e-03, -4.6945e-02],
          [ 3.2648e-02, -1.7021e-03, -2.9392e-02],
          [ 5.8927e-02,  8.1581e-02,  5.5897e-02]],

         [[ 4.5504e-02,  2.1569e-02, -2.1054e-02],
          [ 6.8998e-02,  2.0853e-02, -1.0342e-02],
          [ 8.4306e-02,  9.1999e-02,  6.1293e-02]]],


        [[[-4.3024e-02, -1.2912e-01, -1.7742e-01],
          [-5.7224e-03, -7.1818e-02, -1.0681e-01],
          [ 6.9804e-03, -5.9666e-02, -9.5150e-02]],

         [[-7.5858e-02, -1.6122e-01, -2.0150e-01],
          [-4.4146e-02, -1.0606e-01, -1.3173e-01],
          [-3.3095e-02, -9.2231e-02, -1.2214e-01]],

         [[-5.9760e-02, -1.4226e-01, -1.8205e-01],
          [-2.6412e-02, -8.4385e-02, -1.1003e-01],
          [-2.1240e-02, -6.6071e-02, -9.0027e-02]]],


        [[[ 2.2593e-